In [1]:
import requests
import sqlite3
import json

def fetch_exchange_data(crypto_symbol="BTC", currency="USD"):
    """
    Fetch exchange-specific data for a given cryptocurrency pair using CryptoCompare's API.
    This endpoint returns data on the top exchanges for the specified pair.
    """
    url = "https://min-api.cryptocompare.com/data/top/exchanges/full"
    params = {
        "fsym": crypto_symbol,
        "tsym": currency
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Error occurred: {err}")
    return None

def create_database(db_name="crypto_exchanges.db"):
    """
    Create (or connect to) an SQLite database and create a table to store exchange data.
    """
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS exchange_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            exchange TEXT,
            cryptocurrency TEXT,
            currency TEXT,
            price REAL,
            volume24h REAL,
            timestamp INTEGER
        )
    ''')
    conn.commit()
    return conn

def store_exchange_data(conn, crypto_symbol, currency, exchanges_data):
    """
    Insert exchange data into the database.
    Each entry includes the exchange name, price, 24h volume, and a timestamp.
    """
    c = conn.cursor()
    for exchange in exchanges_data:
        # Each exchange's data dictionary typically includes keys such as PRICE, VOLUME24H, LASTUPDATE, and MARKET.
        price = exchange.get("PRICE")
        volume24h = exchange.get("VOLUME24H")
        last_update = exchange.get("LASTUPDATE")
        exchange_name = exchange.get("MARKET")
        c.execute('''
            INSERT INTO exchange_prices (exchange, cryptocurrency, currency, price, volume24h, timestamp)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (exchange_name, crypto_symbol, currency, price, volume24h, last_update))
    conn.commit()

def main():
    crypto_symbol = "BTC"
    currency = "USD"
    data = fetch_exchange_data(crypto_symbol, currency)
    
    if data and "Data" in data and "Exchanges" in data["Data"]:
        exchanges_data = data["Data"]["Exchanges"]
        conn = create_database()
        store_exchange_data(conn, crypto_symbol, currency, exchanges_data)
        conn.close()
        print("Exchange data stored successfully in the database.")
    else:
        print("Failed to retrieve or parse exchange data.")

if __name__ == "__main__":
    main()


Exchange data stored successfully in the database.


In [ ]:
def view_data(db_name="crypto_exchanges.db"):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Query all data from the exchange_prices table
    cursor.execute("SELECT * FROM exchange_prices")
    rows = cursor.fetchall()
    
    # Print each row in a readable format
    for row in rows:
        print(row)
    
    # Close the connection
    conn.close()

if __name__ == "__main__":
    view_data()


(1, 'Coinbase', 'BTC', 'USD', 95833.05, None, 1740339547)
(2, 'P2PB2B', 'BTC', 'USD', 95806.35, None, 1740339547)
(3, 'cryptodotcom', 'BTC', 'USD', 95833.81, None, 1740339529)
(4, 'Bitstamp', 'BTC', 'USD', 95824.0, None, 1740339537)
(5, 'bullish', 'BTC', 'USD', 95820.6072, None, 1740339529)


In [3]:
import requests
import sqlite3
import time
from datetime import datetime

def get_daily_history(crypto, currency, exchange, limit=365):
    """
    Fetch daily historical price data for a given cryptocurrency on a specific exchange.
    Uses CryptoCompare's histoday endpoint.
    """
    url = "https://min-api.cryptocompare.com/data/v2/histoday"
    params = {
        "fsym": crypto,
        "tsym": currency,
        "limit": limit,
        "e": exchange  # specify the exchange
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if data.get("Response") == "Success":
            return data["Data"]["Data"]
        else:
            print(f"Failed to fetch data for {crypto} on {exchange}: {data.get('Message')}")
            return None
    except Exception as e:
        print(f"Exception for {crypto} on {exchange}: {e}")
        return None

def create_daily_prices_table(db_name="crypto_exchanges.db"):
    """
    Creates (or connects to) an SQLite database and sets up a table for storing daily price data.
    """
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS daily_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            exchange TEXT,
            cryptocurrency TEXT,
            date INTEGER,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volumefrom REAL,
            volumeto REAL
        )
    ''')
    conn.commit()
    return conn

def store_daily_prices(conn, exchange, crypto, daily_data):
    """
    Inserts daily price data for a given cryptocurrency and exchange into the database.
    """
    c = conn.cursor()
    for day in daily_data:
        # Each day's data typically includes: time, high, low, open, close, volumefrom, volumeto.
        c.execute('''
            INSERT INTO daily_prices (exchange, cryptocurrency, date, open, high, low, close, volumefrom, volumeto)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            exchange,
            crypto,
            day['time'],
            day['open'],
            day['high'],
            day['low'],
            day['close'],
            day['volumefrom'],
            day['volumeto']
        ))
    conn.commit()

def main():
    # Hard-coded list of top 10 cryptocurrencies (adjust as needed)
    top_cryptos = ["BTC", "ETH", "USDT", "BNB", "XRP", "ADA", "DOGE", "SOL", "DOT", "LTC"]
    # Exchanges from your output
    exchanges = ['Coinbase', 'P2PB2B', 'cryptodotcom', 'Bitstamp', 'bullish']
    currency = "USD"
    db_name = "crypto_exchanges.db"
    
    conn = create_daily_prices_table(db_name)
    
    for exchange in exchanges:
        for crypto in top_cryptos:
            print(f"Fetching daily history for {crypto} on {exchange}...")
            daily_data = get_daily_history(crypto, currency, exchange, limit=365)
            if daily_data:
                store_daily_prices(conn, exchange, crypto, daily_data)
                print(f"Stored data for {crypto} on {exchange}")
            else:
                print(f"No data for {crypto} on {exchange}")
            # Pause briefly to help avoid hitting API rate limits
            time.sleep(1)
    
    conn.close()
    print("All daily price data stored successfully.")

if __name__ == "__main__":
    main()


Fetching daily history for BTC on Coinbase...
Stored data for BTC on Coinbase
Fetching daily history for ETH on Coinbase...
Stored data for ETH on Coinbase
Fetching daily history for USDT on Coinbase...
Stored data for USDT on Coinbase
Fetching daily history for BNB on Coinbase...
Failed to fetch data for BNB on Coinbase: Coinbase market does not exist for this coin pair (BNB-USD)
No data for BNB on Coinbase
Fetching daily history for XRP on Coinbase...
Stored data for XRP on Coinbase
Fetching daily history for ADA on Coinbase...
Stored data for ADA on Coinbase
Fetching daily history for DOGE on Coinbase...
Stored data for DOGE on Coinbase
Fetching daily history for SOL on Coinbase...
Stored data for SOL on Coinbase
Fetching daily history for DOT on Coinbase...
Stored data for DOT on Coinbase
Fetching daily history for LTC on Coinbase...
Stored data for LTC on Coinbase
Fetching daily history for BTC on P2PB2B...
Stored data for BTC on P2PB2B
Fetching daily history for ETH on P2PB2B...


In [5]:
import sqlite3
import pandas as pd

def export_to_excel(db_name="crypto_exchanges.db", excel_file="crypto_data.xlsx"):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)
    
    # Load all data from the daily_prices table into a DataFrame
    df = pd.read_sql_query("SELECT * FROM daily_prices", conn)
    conn.close()
    
    # Create an Excel writer object using xlsxwriter
    with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
        # Group by exchange and create a sheet for each unique exchange
        for exchange in df['exchange'].unique():
            # Filter the DataFrame for the current exchange
            exchange_df = df[df['exchange'] == exchange]
            # Write the data to a sheet named after the exchange
            exchange_df.to_excel(writer, sheet_name=exchange, index=False)
    
    print(f"Excel file '{excel_file}' has been created with each sheet corresponding to a different exchange.")

if __name__ == "__main__":
    export_to_excel()


Excel file 'crypto_data.xlsx' has been created with each sheet corresponding to a different exchange.
